In [3]:
# Importing additional libraries for statistical analysis and visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from skopt import BayesSearchCV
from math import sqrt
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
import os
from glob import glob
from sklearn.model_selection import TimeSeriesSplit
import joblib

In [16]:
import pandas as pd

path = r'F:\박정현\ML\Mosquito\final\2015_2024_total.xlsx'
df = pd.read_excel(path, engine='openpyxl')

# IQR 계산
Q1_Mosquitoe = df['mosquito'].quantile(0.25)
Q3_Mosquitoe = df['mosquito'].quantile(0.75)
IQR_Mosquitoe = Q3_Mosquitoe - Q1_Mosquitoe
lower_bound_Mosquitoe = Q1_Mosquitoe - 1.5 * IQR_Mosquitoe
upper_bound_Mosquitoe = Q3_Mosquitoe + 1.5 * IQR_Mosquitoe

# 이상치 제거
df_iqr = df[(df['mosquito'] >= lower_bound_Mosquitoe) & (df['mosquito'] <= upper_bound_Mosquitoe)]

# 결과 출력
print(f"Q1: {Q1_Mosquitoe}, Q3: {Q3_Mosquitoe}")
print(f"이상치 제거 전 데이터 개수: {len(df)}, 이상치 제거 후 데이터 개수: {len(df_iqr)}")


Q1: 12.0, Q3: 53.0
이상치 제거 전 데이터 개수: 78909, 이상치 제거 후 데이터 개수: 72171


In [19]:
print(df_iqr.dtypes)


DATE                   datetime64[ns]
DMS_CODE                       object
std_name                       object
TMP                           float64
TMAX                          float64
TMIN                          float64
PCP                           float64
HUM                           float64
WS                            float64
Temp_1_monthly_mean           float64
Temp_1_yearly_mean            float64
DMS_open_1m                     int64
DMS_open_3m                     int64
DMS_open_5m                     int64
DMS_open_10m                    int64
Env_Factors_1m                  int64
Env_Factors_3m                  int64
Env_Factors_5m                  int64
Env_Factors_10m                 int64
landscape                       int64
mosquito                        int64
dtype: object


In [20]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Train-Test 분리
train = df_iqr[df_iqr['DATE'] <= '2023-10-31']
test = df_iqr[df_iqr['DATE'] >= '2024-04-01']

# 날짜(datetime64) 컬럼 제거
if 'DATE' in train.columns:
    train = train.drop(columns=['DATE'])
    test = test.drop(columns=['DATE'])

# 범주형(object) 데이터 제거
train = train.drop(columns=train.select_dtypes(include=['object']).columns)
test = test.drop(columns=test.select_dtypes(include=['object']).columns)

# MinMaxScaler 적용
scaler = MinMaxScaler()
train_scaled = pd.DataFrame(scaler.fit_transform(train), columns=train.columns)
test_scaled = pd.DataFrame(scaler.transform(test), columns=test.columns)

# 독립 변수(X)와 종속 변수(y) 분리
X_train = train_scaled.drop(columns=['mosquito'])
y_train = train_scaled['mosquito']

X_test = test_scaled.drop(columns=['mosquito'])
y_test = test_scaled['mosquito']
